In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

Création des fonctions : 

In [2]:
def A(pN,pM, Xi,m):
    p1 = pN-Xi*(pM-pN)
    pm = pM+Xi*(pM-pN)
    A = []
    for i in range (1,m+1):
        new = p1 + (i-1)*(pm-p1)/m
        A.append(new)
    return A

In [3]:
def S(A): 
    S = []
    for i in range (len(A)):
        for j in range (len(A)): 
            S.append([A[i],A[j]])
    return S
    

In [4]:
def pi1(s): 
    return (s[0]-1)*np.exp(4*(2-s[0]))/(np.exp(4*(2-s[0]))+np.exp(4*(2-s[1]))-1)

In [5]:
def pi2(s): 
    return (s[1]-1)*np.exp(4*(2-s[1]))/(np.exp(4*(2-s[1]))+np.exp(4*(2-s[0]))-1)

In [6]:
def initQwithLargeConstantValue(Q): 
    for i in range (len(Q)):
        for j in range(len(Q[0])):
            Q[i][j]=2
    return Q

In [7]:
#Créer une fonction qui trouve le maximum de Q(s,a)
def trouve_max(Q,s_prime):
    sol = Q[0][s_prime]
    arg = 0
    for i in range (1,len(Q)): 
        if sol <= Q[i][s_prime]:
            sol = Q[i][s_prime]
            arg = i
    return sol, arg        

In [8]:
def epsilon(beta,t):
    #return the value of epsilon at the moment t
    return np.exp(-beta*t)

In [9]:
#current_column_index = index of the price set by the players in t-1
def get_next_action(Q,s_t_index,epsilon,m,trouve_max):
    if np.random.random() < 1-epsilon: 
        return trouve_max(Q,s_t_index)[1]
    else: 
        return np.random.randint(m)

In [10]:
def updateQ(Q,S,a_t_index, s_t_index, alpha, delta, pi, s_t1_index):
    Qbis = Q
    for i in range(len(Q)): 
        for j in range(len(Q[0])):
            if i == a_t_index and j == s_t_index: #faire attention à différencier valeurs et indices : ici on parle d'indice
                Qbis[i][j]= (1-alpha)*Q[i][j] + alpha*(pi(S[s_t1_index]) + delta*trouve_max(Q,s_t1_index)[0])
    return Qbis

In [11]:
def find_index(S,s_t): 
    index = -1
    for test in range (len(S)):
        if abs(S[test][0] - s_t[0]) <= 10**(-100) and abs(S[test][1] - s_t[1]) <= 10**(-100) :
            index = test 
    return index

In [12]:
def jeu(pN,pM,Xi,m, A, S, epsilon, beta, alpha, delta, initQ, updateQ, nb_iteration, get_next_action,trouve_max,pi1,pi2, find_index):
    #Initialisation
    A = A(pN,pM, Xi,m)
    S = S(A)
    Q1 = np.zeros((len(A),len(S))) #Q matrice j1
    Q1 = initQ(Q1)
    Q2 = np.zeros((len(A),len(S))) #Q matrice j2
    Q2 = initQ(Q2)
    t = 0
    s_t_index = np.random.randint(m*m)
    #Phase itérative
    while t < nb_iteration:
        a_t_index1 = get_next_action(Q1,s_t_index,epsilon(beta,t),m,trouve_max)
        a_t_index2 = get_next_action(Q2,s_t_index,epsilon(beta,t),m,trouve_max)
        #print(a_t_index1,a_t_index2)
        s_t1 = [A[a_t_index1],A[a_t_index2]]
        s_t1_index = find_index(S,s_t1)
        Q1 = updateQ(Q1,S, a_t_index1, s_t_index, alpha, delta, pi1, s_t1_index)
        #print(Q1[a_t_index1,s_t_index])
        Q2 = updateQ(Q2,S,a_t_index2, s_t_index, alpha, delta, pi2, s_t1_index)
        t = t+1
        s_t_index = s_t1_index
    return Q1, Q2, S[s_t1_index]
    

Test des différentes fonctions

In [13]:
A_test = A(1.78,2,0.1,15)
print(A_test)

[1.758, 1.7756, 1.7932, 1.8108, 1.8284, 1.8459999999999999, 1.8636, 1.8812, 1.8987999999999998, 1.9163999999999999, 1.934, 1.9515999999999998, 1.9691999999999998, 1.9868, 2.0044]


In [14]:
len(A_test)

15

In [15]:
S_test = S(A_test)
print(S_test)

[[1.758, 1.758], [1.758, 1.7756], [1.758, 1.7932], [1.758, 1.8108], [1.758, 1.8284], [1.758, 1.8459999999999999], [1.758, 1.8636], [1.758, 1.8812], [1.758, 1.8987999999999998], [1.758, 1.9163999999999999], [1.758, 1.934], [1.758, 1.9515999999999998], [1.758, 1.9691999999999998], [1.758, 1.9868], [1.758, 2.0044], [1.7756, 1.758], [1.7756, 1.7756], [1.7756, 1.7932], [1.7756, 1.8108], [1.7756, 1.8284], [1.7756, 1.8459999999999999], [1.7756, 1.8636], [1.7756, 1.8812], [1.7756, 1.8987999999999998], [1.7756, 1.9163999999999999], [1.7756, 1.934], [1.7756, 1.9515999999999998], [1.7756, 1.9691999999999998], [1.7756, 1.9868], [1.7756, 2.0044], [1.7932, 1.758], [1.7932, 1.7756], [1.7932, 1.7932], [1.7932, 1.8108], [1.7932, 1.8284], [1.7932, 1.8459999999999999], [1.7932, 1.8636], [1.7932, 1.8812], [1.7932, 1.8987999999999998], [1.7932, 1.9163999999999999], [1.7932, 1.934], [1.7932, 1.9515999999999998], [1.7932, 1.9691999999999998], [1.7932, 1.9868], [1.7932, 2.0044], [1.8108, 1.758], [1.8108, 1.77

In [17]:
print([S_test[90],pi1(S_test[90]),pi2(S_test[90])])

[[1.8636, 1.758], 0.44375368876753135, 0.5942162678998217]


In [18]:
Q = np.zeros((len(A_test),len(S_test)))
Q = initQwithLargeConstantValue(Q)
print(Q)

[[2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]


In [19]:
Q = [[1,2,3],[4,5,6],[7,8,9]]
trouve_max(Q,0)

(7, 2)

In [20]:
for t in range (10): 
    print(epsilon(10**(-5),t))

1.0
0.9999900000499998
0.9999800001999987
0.9999700004499955
0.9999600007999894
0.9999500012499791
0.999940001799964
0.9999300024499428
0.9999200031999147
0.9999100040498785


In [21]:
Q = [[1,2,3],[4,5,6],[7,8,9]]
print(get_next_action(Q,1,0.5,15,trouve_max))

2


In [22]:
Q = np.zeros((len(A_test),len(S_test)))
Q = initQwithLargeConstantValue(Q)
#print(updateQ(Q,S_test,2,2, 0.1, 0.95, pi, np.random.randint(225))) #Ca change bien quand on change s_t1_index

In [23]:
#for i in range (len(S)):
    #print(i,find_index(S,S[i])) 
for i in range (len(S_test)):
    if i != find_index(S_test,S_test[i]):
        print(i,find_index(S_test,S_test[i])) 
        print(S_test[i],S_test[find_index(S_test,S_test[i])]) #on doit rien print 

In [25]:
jeu(pN=1.78,pM=2,Xi=0.1,m=15, A=A, S=S, epsilon = epsilon, beta=10**(-5), alpha = 0.1, delta = 0.95, initQ=initQwithLargeConstantValue, updateQ=updateQ, nb_iteration=100000, get_next_action = get_next_action,trouve_max=trouve_max,pi1=pi1, pi2=pi2, find_index=find_index)

(array([[5.47892629, 4.78780959, 3.41921904, ..., 4.29689574, 6.36281308,
         9.34025485],
        [5.77676828, 4.62612108, 3.67034834, ..., 3.84711381, 5.63583742,
         7.19946042],
        [3.90244426, 8.74131865, 4.15904209, ..., 8.90378285, 5.618535  ,
         5.63992645],
        ...,
        [5.40281463, 5.80440727, 8.94548117, ..., 5.16801768, 5.35726577,
         7.01249417],
        [5.70227958, 5.47306089, 5.65503001, ..., 4.79834188, 5.78553448,
         6.96294491],
        [5.86466908, 4.36790852, 4.36827573, ..., 4.6134248 , 6.80497954,
         7.02119358]]),
 array([[5.46793584, 3.50569244, 4.29194977, ..., 3.90051611, 5.34684121,
         7.25386752],
        [6.18796115, 8.68697965, 4.18564052, ..., 5.17929853, 5.94054156,
         6.29491596],
        [5.74812122, 5.26392662, 3.44723012, ..., 4.79135112, 5.67170156,
         6.66586655],
        ...,
        [6.21939947, 4.66008651, 3.50290384, ..., 5.1642587 , 5.17725198,
         6.57619348],
        [5.7

Pour 
- 100 rep : < 1sec
- 1000 rep : < 1 sec
- 10000 rep : env. 4 sec
- 100000 rep : env. 40 sec
- 1000000 rep : env. 7mn